# Decorators

a **decorator** function:

- takes a function as an argument
- returns a closure
- the closure usually accepts any combination of parameters
- runs some code in the inner function (closure)
- calls the original function using the arguments pas to the closure
- return whatever is returned by that function call


In [7]:
from functools import wraps


def counter(fn):
    count = 0

    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        return fn(*args, **kwargs)
    # inner = wraps(fn)(inner)
    return inner


In [8]:
@counter
def mul(a, b, c=1):
    return a * b * c


help(mul)


Help on function mul in module __main__:

mul(a, b, c=1)



# Decorator Application (Timing)


In [9]:
def timed(fn):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start

        args_ = [str(a) for a in args]
        kwargs_ = [f"{k}={v}" for k, v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)

        print(f"{fn.__name__}({args_str}) took {elapsed:.6f} to run")
        return result
    return inner


In [13]:
def calc_recursive_fib(n):
    if n <= 2:
        return 1
    else:
        return calc_recursive_fib(n-1) + calc_recursive_fib(n-2)


@timed
def fib_recursive(n):
    return calc_recursive_fib(n)


In [14]:
fib_recursive(3)


fib_recursive(3) took 0.000002 to run


2

In [15]:
fib_recursive(6)


fib_recursive(6) took 0.000004 to run


8

In [21]:
fib_recursive(36)


fib_recursive(36) took 2.158495 to run


14930352

In [22]:
@timed
def fib_loop(n):
    fib_1 = fib_2 = 1
    for _ in range(3, n+1):
        fib_2, fib_1 = fib_1 + fib_2, fib_2
    return fib_2


In [23]:
fib_loop(6)


fib_loop(6) took 0.000002 to run


8

In [24]:
fib_loop(36)


fib_loop(36) took 0.000005 to run


14930352

In [25]:
fib_loop(1)


fib_loop(1) took 0.000002 to run


1

In [26]:
from functools import reduce


@timed
def fib_reduce(n):
    initial = (1, 0)
    dummy = range(n)
    fib_n = reduce(lambda prev, n: (
        prev[0] + prev[1], prev[0]), dummy, initial)
    return fib_n[0]


fib_reduce(36)


fib_reduce(36) took 0.000008 to run


24157817

# Decorator application (Logger, Stacked decorators)


In [27]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone

    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print(f"{run_dt}: called {fn.__name__}")
        return result
    return inner


In [28]:
@logged
def func_1():
    pass


In [29]:
func_1()


2023-02-15 10:17:51.548595+00:00: called func_1


In [32]:
@logged
@timed
def fact(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))


In [33]:
fact(3)


fact(3) took 0.000017 to run
2023-02-15 10:23:11.967151+00:00: called fact


6

In [36]:
def fact(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))


fact = logged(timed(fact))


In [37]:
fact(4)


fact(4) took 0.000007 to run
2023-02-15 10:25:19.529682+00:00: called fact


24

# Decorator Application (Memoization)


In [38]:
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)


In [40]:
fib(10)


Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating

55

In [41]:
class Fib:
    def __init__(self) -> None:
        self.cache = {1: 1, 2: 1}

    def fib(self, n):
        if n not in self.cache:
            print(f'Calculating fib({n})')
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]


In [42]:
f = Fib()
f.fib(10)


Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


55

In [47]:
def fib():
    cache = {1: 1, 2: 1}

    def calc_fib(n):
        if n not in cache:
            print(f'Calculating fib({n})')
            cache[n] = calc_fib(n-1) + calc_fib(n-2)
        return cache[n]
    return calc_fib


f = fib()
f(10)


Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


55

In [49]:
def memoize_fib(fib):
    cache = {1: 1, 2: 1}

    def inner(n):
        if n not in cache:
            cache[n] = fib(n)
        return cache[n]
    return inner


@memoize_fib
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)


In [50]:
fib(10)


Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


55

In [51]:
def memoize(fn):
    cache = dict()

    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    return inner


@memoize
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)


In [52]:
fib(10)


Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


55

In [53]:
def fact(n):
    print(f'Calculating fact({n})')
    return 1 if n < 2 else n * fact(n-1)


In [54]:
fact(6)


Calculating fact(6)
Calculating fact(5)
Calculating fact(4)
Calculating fact(3)
Calculating fact(2)
Calculating fact(1)


720

In [55]:
@memoize
def fact(n):
    print(f'Calculating fact({n})')
    return 1 if n < 2 else n * fact(n-1)


In [56]:
fact(6)


Calculating fact(6)
Calculating fact(5)
Calculating fact(4)
Calculating fact(3)
Calculating fact(2)
Calculating fact(1)


720

In [57]:
fact(6)


720

In [58]:
from functools import lru_cache


@lru_cache()
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)


In [59]:
fib(10)


Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


55

In [60]:
fib(11)


Calculating fib(11)


89

In [63]:
@lru_cache(maxsize=8)  # None: unlimited cache
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)


In [64]:
fib(8)


Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


21

In [65]:
fib(8)


21

In [66]:
fib(16)


Calculating fib(16)
Calculating fib(15)
Calculating fib(14)
Calculating fib(13)
Calculating fib(12)
Calculating fib(11)
Calculating fib(10)
Calculating fib(9)


987

In [ ]:
# call function 10 times

def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(10):
            start = perf_counter()
            result = fn(*args, **kwargs)
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / 10
        print(avg_elapsed)
        return result
    return inner


In [ ]:
def timed(fn, reps):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / reps
        print(avg_elapsed)
        return result
    return inner


my_func = timed(my_func, 10)  # worked


@timed(10)
def my_func():  # not work
    ...


**timed(10)** will need to **return** our original **timed** decorator when **called**


In [ ]:
def outer(reps):
    def timed(fn):
        from time import perf_counter

        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                total_elapsed += (perf_counter() - start)
            avg_elapsed = total_elapsed / reps
            print(avg_elapsed)
            return result
        return inner

    return timed


outer function is not itself a decorator -> it returns a decorator when called (Decorator Factories)